In [6]:
import scripts
import data_manager.utils as sbtu

[1, 2]


[[1, 1, 2], [1, 2, 1], [2, 1, 1]]

In [7]:
(
    _price_data,
    _bench,
    _strategy_overview,
    _peak_table,
    _entry_table,
    _strategy_lookup
) = scripts.load_data_package()
_relative_rebased = sbtu.PriceGlob(_price_data).relative_rebased(_bench.close)
_price_data_by_symbol = sbtu.PriceGlob(_price_data).swap_level()

Counter({3: 1, 1: 1, 2: 1})

In [ ]:
from dataclasses import dataclass

"""
Position: combination of system calculation and actual fill data
symbol | size | entry_price | stop_price |
sys    | sys  | broker      | sys        |
"""

# @dataclass
# class Position:
#     symbol: str
#     size: float
#     entry_price:
#     def __init__(self, symbol, size):
#         self.symbol = symbol
#         self.size = size






In [2]:
"""
TBD need to know if position risk has been reduced, otherwise don't generate orders for symbol

Option 2:
    - Store initial_size and current_size of position
    - if current_size is some criteria below the initial size, then risk is reduced
    - filter out stocks where criteria is not met

Position Table Generation:
1. Each row in the position table shall obtain the following info:
    - Symbol name
    - initial size
    - entry date?
    - direction?
    -

Order Table Generation:
1.) Order Table generation shall follow this procedure:
    1. Retrieve data for current open positions
    2. For each open position:
        1. if at least one of the following is true:
            TRAIL_STOP || STOP || TARGET_EXIT || FRENCH_STOP || REGIME_END
            1. Generate an order entry, where type is the first discovered reason for entry creation
    3. For each symbol, create order entry if:
        symbol not in order_table && risk_reduced is True
    4.


Order Table Output:
1.) The following data shall be stored in the order_table, sorted by close type first. It may be a view of a larger table:
    Symbol,
    direction of trade,
    position size (dollars or shares),
    type: [
        ENTRY,
        TRAIL_STOP,
        STOP,
        TARGET_EXIT,
        FRENCH_STOP,
        REGIME_END
    ]


2.) Output of the order table shall be sorted by type, with the following priority:
    1. STOP
    2. TRAIL_STOP
    3. TARGET_EXIT
    4. FRENCH_STOP
    5. REGIME_END
    6. ENTRY



2.)
Entries/Exits in the order table shall be generated from
"""


"\nTBD need to know if position risk has been reduced, otherwise don't generate orders for symbol\n\nOption 2:\n    - Store initial_size and current_size of position\n    - if current_size is some criteria below the initial size, then risk is reduced\n    - filter out stocks where criteria is not met\n\nPosition Table Generation:\n1. Each row in the position table shall obtain the following info:\n    - Symbol name\n    - position size\n    - direction?\n\nOrder Table Generation:\n1.) Order Table generation shall follow this procedure:\n    1. Retrieve data for current open positions\n    2. For each open position:\n        1. if at least one of the following is true:\n            TRAIL_STOP || STOP || TARGET_EXIT || FRENCH_STOP || REGIME_END\n            1. Generate an order entry, where type is the first discovered reason for entry creation\n    3. For each symbol, create order entry if:\n        symbol not in order_table && risk_reduced is True\n    4.\n\n\nOrder Table Output:\n1.) 

In [19]:

for symbol, glob in _price_data_by_symbol.items():
    symbol_data = _strategy_lookup[symbol]
    rg_id = symbol_data.regime_table.rg.iat[-1]
    rg_swings = symbol_data.peak_table.loc[
        (symbol_data.peak_table.type == rg_id)
    ]
    latest_swing = rg_swings.iloc[-1]


''

In [37]:
def insert_elem(arr, pos, elem):
    """
    shift values from pos... to the left
    insert elem at pos,
    """
    # shift values first
    tmp = arr[pos]
    for i in range(pos, len(arr)):
        num = tmp
        if num is None:
            break
        tmp = arr[i+1]
        arr[i+1] = num
    arr[pos] = elem


def merge_arrays(arr1, arr2):
    for num in arr2:
        start = 0 # temporary
        end = len(arr1) - 1

        # mid should be the insert position
        # if final compare is num < arr1[mid]
        #   insert = mid
        # else if final compare is >= arr1[mid]
        #   insert = mid + 1
        insert_pos = -1
        while start <= end:
            mid = (start + end) // 2
            if arr1[mid] is None or num < arr1[mid]:
                end = mid - 1
                insert_pos = mid
            elif num >= arr1[mid]:
                start = mid + 1
                insert_pos = mid + 1

        insert_elem(arr1, insert_pos, num)
    return arr1

merge_arrays([1,4,9,14,None, None, None, None], [2,3,8,12])

[1, 2, 3, 4, 8, 9, 12, 14]